# Solving the Minimal Surface Problem: Discretization and Implementation

In the last post, we transformed the classical Plateau problem into a **convex optimization problem** over differential forms:

$$
\min_{\eta \in \Omega^1(M), \ d\eta = \delta_\Gamma} \| \eta \|_{\text{mass}}
$$

This was a major step—we went from geometry to a robust, mesh-free formulation. Now, it's time to **discretize** the problem and **build a solver**.

---

## 1. Recap: What Are We Solving?

We’re looking for a 1-form $\eta$ (a vector field on a volume) such that:
- It satisfies the constraint $d\eta = \delta_\Gamma$ (its exterior derivative equals the input curve)
- It minimizes the **mass norm** (an $L^1$-like norm encouraging minimal area)

Our computational domain is a cube $[0, 1)^3$ with **periodic boundaries**, so we can use **FFT-based solvers**.

---

## 2. Discretizing the Domain

Let’s discretize the domain into a regular grid:


In [ ]:
nx, ny, nz = 64, 64, 64
h = 1.0 / nx  # grid spacing


We store the 1-form $\eta$ as a vector field:


In [ ]:
# Vector field on grid vertices: shape (nx, ny, nz, 3)
import numpy as np
eta = np.zeros((nx, ny, nz, 3))


---

## 3. Encoding the Boundary Curve

We encode the input curve $\Gamma$ as a **Dirac-δ 2-form**, represented as a sharply concentrated vector field. In practice, we rasterize the curve onto the grid:


In [ ]:
delta_gamma = np.zeros((nx, ny, nz, 3))

# For each segment of Γ, assign vector value to nearby grid cells
# Use smoothed delta kernels (e.g., Gaussian blob or uniform sphere)


---

## 6. Poisson Solver via FFT

To solve $\Delta \phi = \text{rhs}$:


In [ ]:
def poisson_fft(rhs, h):
    nx, ny, nz = rhs.shape
    kx = np.fft.fftfreq(nx) * 2 * np.pi
    ky = np.fft.fftfreq(ny) * 2 * np.pi
    kz = np.fft.fftfreq(nz) * 2 * np.pi
    KX, KY, KZ = np.meshgrid(kx, ky, kz, indexing='ij')

    lap = 4 * (np.sin(KX / 2)**2 + np.sin(KY / 2)**2 + np.sin(KZ / 2)**2)
    rhs_hat = np.fft.fftn(rhs)

    phi_hat = np.zeros_like(rhs_hat)
    phi_hat[lap != 0] = rhs_hat[lap != 0] / lap[lap != 0]
    phi_hat[lap == 0] = 0  # mean-zero constraint

    phi = np.fft.ifftn(phi_hat).real
    return phi


---

## 7. Shrinkage Step (Proximal Operator)

This solves:

$$
\min_X \| X \|_1 + \frac{\tau}{2} \| X - V \|_2^2
$$

The solution is **soft-thresholding**:


In [ ]:
def shrink(V, tau):
    norm = np.linalg.norm(V, axis=-1, keepdims=True)
    factor = np.maximum(1 - 1 / (tau * norm), 0)
    return factor * V